In [1]:


import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


## load artifacts

In [32]:


#model
model = load_model('model.h5')

#scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

#encodings gender
with open('label_encoder_gender.pkl', 'rb') as f:
    gender_encodings = pickle.load(f)
    

#encodings age
with open('onehot_encoder_geo.pkl', 'rb') as f:
    geo_encodings = pickle.load(f)


In [56]:
# example input data
input_data = {
    'credit_score': 700,
    'Geography': 'France',
    'Gender': 'Female',
    'age': 30,
    'Tenure': 5,
    'balance': 1000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,
    
} 

In [33]:
print(type(geo_encodings))

<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [37]:
#OHE for geography
geo_encoded = geo_encodings.transform([[input_data['Geography']]]).toarray()

# geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo_encodings.get_feature_names_out(['Geography']))

geo_encoded

c:\Users\user\Documents\Deep Learning projects\ANN Project\myenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [38]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo_encodings.get_feature_names_out(['Geography']))


In [39]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [64]:
#combine OHE columns with entire data
# Convert input_data dict to DataFrame
input_df = pd.DataFrame([input_data])

# Combine with geo_encoded_df
final_data = pd.concat([input_df.reset_index(drop=True).drop(columns=['Geography']), geo_encoded_df], axis=1)

In [65]:
final_data

,credit_score,Gender,age,Tenure,balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,Female,30,5,1000,2,1,1,50000,1.0,0.0,0.0


In [66]:
#Encode the gender column

final_data['Gender'] = gender_encodings.fit_transform(final_data['Gender'])

In [78]:
final_data

,credit_score,Gender,age,Tenure,balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,0,30,5,1000,2,1,1,50000,1.0,0.0,0.0


In [77]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit_score       1 non-null      int64  
 1   Gender             1 non-null      int64  
 2   age                1 non-null      int64  
 3   Tenure             1 non-null      int64  
 4   balance            1 non-null      int64  
 5   NumOfProducts      1 non-null      int64  
 6   HasCrCard          1 non-null      int64  
 7   IsActiveMember     1 non-null      int64  
 8   EstimatedSalary    1 non-null      int64  
 9   Geography_France   1 non-null      float64
 10  Geography_Germany  1 non-null      float64
 11  Geography_Spain    1 non-null      float64
dtypes: float64(3), int64(9)
memory usage: 228.0 bytes


In [75]:
## Data scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(final_data)

In [79]:
X

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [81]:
#predictions
predictions = model.predict(X)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
[[0.23553362]]


In [83]:
# Prediction probability
predictions[0][0]

np.float32(0.23553362)

In [84]:
if predictions[0][0] > 0.5:
    print("Prediction: Positive")
else:
    print("Prediction: Negative")

Prediction: Negative
